# Session 1: Web Browsing Traces
## Introduction
The topic of this session is **Digital Traces and Human Behavior**. We will provide a dataset of individuals' web browsing traces combined with their sociodemographic information collected via surveys.

In this session,we will analyze the data to examine the patterns of individuals' internet use and how they vary across different demographic groups.

### Learning Objectives:
* Extract patterns of internet use from browsing traces.
* Examine how browsing patterns differ across demographic groups.

---
### Steps:
1. **Data Cleaning & Setup**: Preparing the raw browsing logs.
2. **Individual profile**: Profiling a single individual's "Digital Profile".
3. **Group Project**:
    * **Task 1**: Classify the 50 users based on browsing rhythms.
    * **Task 2**: Connect behaviors to demographics (Gender, Age, Politics).




### Programming Toolkit
To perform this analysis, we will use several specialized Python libraries. You can refer to the official documentation for detailed API information:

| Library | Category | Purpose | Documentation |
| :--- | :--- | :--- | :--- |
| **Pandas** | Data Processing | Data manipulation and DataFrame operations | [Docs](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) |
| **NumPy** | Mathematics | Numerical computing and array operations | [Docs](https://numpy.org/doc/stable/user/index.html#user) |
| **Matplotlib** | Visualization | Core library for static plots | [Docs](https://matplotlib.org/stable/contents.html) |
| **Seaborn** | Visualization | Statistical data visualization | [Docs](https://seaborn.pydata.org/tutorial/introduction.html) |
| **WordCloud** | NLP/Visualization | Generating tag clouds from text | [Docs](https://amueller.github.io/word_cloud/) |
| **Circlify** | Visualization | Circle packing for bubble charts | [GitHub](https://github.com/elmotec/circlify) |
| **Stop-Words** | NLP | Filtering common words in multiple languages | [PyPI](https://pypi.org/project/stop-words/) |
| **Scipy.stats** | Statistics | Performing t-tests to compare groups. |[Docs](https://docs.scipy.org/doc/scipy/reference/stats.html)|

In [ ]:
# Install specialized libraries
!pip install circlify wordcloud stop-words

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import circlify
import re
import os
import warnings
from collections import Counter
from wordcloud import WordCloud
from stop_words import get_stop_words
from urllib.parse import unquote
from scipy import stats

warnings.filterwarnings('ignore')

# Plotting configuration
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline


###Dataset Descriptions

| Dataset | Granularity | Key Columns | Description |
| :--- | :--- | :--- | :--- |
| **Browsing traces** | High-frequency | `pid`, `used_at`,`url`,`domain`, `duration`, `wave` | Raw web traces recording every URL visited. |
| **Demographics** | Static | `pid`, `gender`, `age_group`, `politics` | Constant user characteristics. |
| **Domain Map** | Metadata | `domain`, `category` | Mapping URLs to functional categories (e.g., Social, News). |

For privacy and sensitivity reasons, we removed all URLs except search-engine queries, which were kept for the word cloud analysis.


#### **Demographic Coding**
* **Gender**: `1` = Male, `0` = Female.
* **Politics**: `1` = Left, `0` = Right.



In [ ]:

BASE_URL = "https://raw.githubusercontent.com/Yajing-W/CSS_winterschool_practice/refs/heads/main/"

df = pd.read_parquet(BASE_URL + "browsing_data.parquet")
demo_df = pd.read_csv(BASE_URL + "demographics.csv")
category_df = pd.read_csv(BASE_URL + "domain_category_map.csv")
print("Datasets loaded successfully.")

print(f"Browsing traces: {df.shape}")
print(f"Demographics: {demo_df.shape}")
print(f"Domain_category_map: {category_df.shape}")

## Part 1: Data Cleaning (Valid Signal Identification)

Raw browsing logs often contain noise such as background ads, automatic refreshes, or idle sessions. Before profiling users, we want to ensure the data represents genuine human intent and active engagement.

> * When converting time, use `pd.to_datetime(..., utc=True)` to handle potential timezone issues efficiently.
> * **Filtering**: In real-world datasets, a "zero duration" is often a tracking error or a failed page load.

#### Basic Cleaning & Anomaly Detection
1. **Time Conversion**: Convert the `used_at` column to Pandas `datetime` objects.
2. **Standard Filter**: Remove records where `duration` is 0 or missing.
3. **Ghost Session Identification**: Identify "Ghost Sessions" where `duration` exceeds 4 hours (14,400s). These are typically caused by tabs left open overnight or background processes (e.g., music/gaming).

In [ ]:
# Convert to datetime and ensure UTC consistency
df['used_at'] = pd.to_datetime(df['used_at'], utc=True)

# Remove zero/null durations
initial_count = len(df)
df = df[df['duration'] > 0].dropna(subset=['duration'])
print(f"Standard Cleaning: Removed {initial_count - len(df)} invalid records.")

# Anomaly Detection (Ghost Sessions)
# Define threshold: 4 hours = 14400 seconds
GHOST_THRESHOLD = 14400
df['is_ghost'] = df['duration'] > GHOST_THRESHOLD

ghost_count = df['is_ghost'].sum()
ghost_time_share = df[df['is_ghost']]['duration'].sum() / df['duration'].sum()

print(f"Ghost Detection: Found {ghost_count} sessions > 4 hours.")

# Create a cleaned dataset for behavior analysis
df_final = df[df['is_ghost'] == False].copy()

plt.figure(figsize=(12, 5))

# Plot: Boxplot comparison
plt.subplot(1, 2, 1)
sns.boxplot(data=df, y=df['duration'] / 60)
plt.title("Before Cleaning")
plt.ylabel("Duration (Minutes)")

plt.subplot(1, 2, 2)
sns.boxplot(data=df_final, y=df_final['duration'] / 60)
plt.title("After Cleaning")
plt.ylabel("Duration (Minutes)")

plt.tight_layout()
plt.show()

## Part 1.2: Behavioral Profiling & Visualization (EDA)

With the noise removed, we can now answer: **"How actively do our users engage online?"** We will use three classic visualization methods to understand the population.

#### Quantifying User Engagement
1. **Active Days**: How many unique days did each user (`pid`) appear during the month?
2. **Daily Usage**: On the days they were active, how many hours did they spend browsing on average?
3. **Distribution Analysis**:
    * Use a **Histogram** to see the spread of active days.
    * Use a **CDF (Cumulative Distribution Function)** to identify the "Heavy Users".

    

In [ ]:

df_final['date'] = df_final['used_at'].dt.date

# Calculate Active Days
active_days = df_final.groupby('pid')['date'].nunique()

# Calculate Daily Intensity (Hours per day per user)
daily_usage = df_final.groupby(['pid', 'date'])['duration'].sum() / 3600
avg_daily_hours = daily_usage.groupby('pid').mean()

# 2. Visualizing User Distribution
plt.figure(figsize=(15, 5))

# A. Histogram: Active Days
plt.subplot(1, 3, 1)
sns.histplot(active_days, bins=15, color='skyblue', kde=True)
plt.title("User Persistence\n(Active Days Distribution)")
plt.xlabel("Days Active")

# B. Histogram: Avg Daily Hours
plt.subplot(1, 3, 2)
sns.histplot(avg_daily_hours, bins=20, color='salmon', kde=True)
plt.title("Usage Intensity\n(Avg Hours/Day)")
plt.xlabel("Hours")

# C. CDF Plot
plt.subplot(1, 3, 3)
sns.ecdfplot(avg_daily_hours, color='green', lw=2)
plt.axhline(0.8, color='grey', linestyle='--', alpha=0.7, label='80%')
plt.axvline(avg_daily_hours.quantile(0.8), color='orange', linestyle='--')
plt.title("Intensity CDF")
plt.xlabel("Avg Hours/Day")
plt.legend()

plt.tight_layout()
plt.show()

# 3. Final Summary Stats
print(f"Summary Statistics:")
print(f"- Average Active Days: {active_days.mean():.1f} days")
print(f"- Median Daily Usage: {avg_daily_hours.median():.2f} hours")
print(f"- 80% of users spend less than {avg_daily_hours.quantile(0.8):.2f} hours per day.")

### Why these plots?
1. Tells us how "regular" our users are. If most users only appear for 1-2 days, our conclusions about their "habits" might be weak. We want to see a spread that shows consistent engagement.
2. Understanding the "average" and the "extreme".
3. The CDF helps us answer: "What defines a 'Heavy User'?"
    * For example, the 80% line allows us to see the threshold where the top 20% of users reside.

4. Outlier Detection: If a user browses 20 hours a day, are they a human or a bot?


## Part 2: Feature Engineering (Temporal & Categorical)

We need to transform low-level logs into high-level behavioral features. By breaking down timestamps and mapping URLs to categories, we can understand *when* people are active and *what* they are doing.

#### Temporal Features
* **Action**: Extract `hour` and `day_name` from `used_at`.


#### Content Categorization
* **Action**: Map each record to a category.

> **Tip:**
>We use a `left merge` to ensure every browsing record is kept.
> Since both dataframes now share the same column name (`domain`), we can use `pd.merge(..., on='domain')`. Always check for `NaN` values in the new `category` column to ensure your mapping coverage is sufficient.
> If a domain isn't in your map, use `.fillna('uncategorized')` to ensure your analysis remains complete.

> **Reference:**
> In professional settings, we could use services like [Webshrinker](https://webshrinker.com/) to automatically categorize millions of domains using machine learning and domain database lookups.

In [ ]:
# --- TEMPORAL FEATURE EXTRACTION ---
# Extract hour and day name from the timestamp
df_final['hour'] = df_final['used_at'].dt.hour
df_final['day_name'] = df_final['used_at'].dt.day_name()

# --- CATEGORY MAPPING ---
# Perform a left join to enrich the browsing data with website categories
# Both dataframes use 'domain' as the join key
df_final = pd.merge(
    df_final,
    category_df,
    on='domain',
    how='left'
)

# Fill domains not found in the category map with 'uncategorized'
df_final['category'] = df_final['category'].fillna('uncategorized')

# --- DATA VALIDATION (Print Statistics) ---

# Ensure no users (pids) were lost during the merge
unique_pids = df_final['pid'].nunique()
print(f"Total Unique Users (pid) retained: {unique_pids}")
print(f"Total Records in table: {len(df_final)}")

# Preview the enriched table
print("\n--- Feature Table Preview ---")
# Focus on the relationship between pid, time, and category
display(df_final[['pid', 'used_at', 'hour', 'day_name', 'domain', 'category']].head())

# Categorization Summary
print("\n--- Category Coverage Summary ---")
cat_counts = df_final['category'].value_counts()
print(cat_counts)

# Verification Visualization: Quick check of data distribution
plt.figure(figsize=(10, 4))
cat_counts.head(10).sort_values().plot(kind='barh', color='skyblue')
plt.title("Distribution of Top 10 Categories")
plt.xlabel("Number of Records")
plt.show()

## Part 3: Behavioral Visualization

Each user has a unique "browsing profile"— an image of when and how they interact with the web. By visualizing the **7x24 hour rhythm** of a specific user, we can uncover their personal habits.

#### The Weekly Rhythm (Heatmap)
We will create a heatmap that shows activity intensity across different hours and days for a **single user**.

> **Tip:**
> A "Global Heatmap" (all users) often washes out individual traits. Filtering for a specific `pid` allows us to see true human behavior—like consistent late-night browsing on weekends vs. structured work hours on weekdays.


We will define a function `plot_weekly_heatmap` to make this visualization reusable for any user in the dataset.

In [ ]:
def plot_weekly_heatmap(df, user_id):
    """
    Generates a 7x24 heatmap showing browsing density for a specific user.
    """
    # Filter data for the specific user
    user_df = df[df['pid'] == user_id].copy()

    # 2Define the logical order of days for the Y-axis
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    hours_order = list(range(24))

    # Index: Days of the week, Columns: Hours of the day
    # We count the number of sessions (using 'pid' as a proxy for count)
    pivot_table = user_df.pivot_table(
        index='day_name',
        columns='hour',
        values='pid',
        aggfunc='count'
    ).reindex(days_order)

    # Fill hours with no activity with 0
    pivot_table = pivot_table.fillna(0)
    pivot_table = pivot_table.reindex(index=days_order, columns=hours_order, fill_value=0)
    # 5. 可视化

    # Visualize
    plt.figure(figsize=(24, 6))
    sns.heatmap(
        pivot_table,
        cmap="YlGnBu",
        annot=False,
        cbar_kws={'label': 'Activity Intensity (Session Count)'}
    )

    plt.title(f"Weekly Browsing Density Heatmap: User {user_id}", fontsize=14, fontweight='bold')
    plt.xlabel("Hour of Day (0-23)")
    plt.ylabel("")
    plt.show()


In [ ]:
# --- CALL THE FUNCTION ---

# Find the most active user to ensure we have an interesting result
top_user_id = df_final['pid'].value_counts().idxmax()
print(f"Analyzing Digital Fingerprint for Top User: {top_user_id}")

# Run the function
plot_weekly_heatmap(df_final, top_user_id)

#### Top Websites (Bar Charts)
Now that we know *when* this user is active, let's see *where* they go. We will identify the most frequently visited domains for the same user.

#### Online Activities Overview (Bubble Chart)
To get a high-level view of this user's interests, we will use a **Bubble Chart**. Unlike a standard bar chart, a bubble chart represents the frequency of visits per category through the area of a circle, providing a more intuitive sense of "interest density."

> **Insight:**
> Comparing specific domains with general categories helps us distinguish between a user's **tools** (e.g., Google, GitHub) and their **hobbies** (e.g., Entertainment, Social Media).

In [ ]:
import circlify

def plot_top_websites(df, user_id, top_n=10):
    """
    Plots a horizontal bar chart of the top N most visited domains for a specific user.
    """
    user_df = df[df['pid'] == user_id].copy()
    top_sites = user_df['domain'].value_counts().head(top_n)

    if top_sites.empty:
        print(f"No data found for User: {user_id}")
        return

    plt.figure(figsize=(10, 5))
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, top_n))
    plt.barh(top_sites.index, top_sites.values, color=colors)
    plt.gca().invert_yaxis()
    plt.title(f"Top {top_n} Most Visited Websites (User: {user_id})", fontsize=13, fontweight='bold')
    plt.xlabel("Visit Count")
    plt.show()

def plot_activity_bubbles(df, user_id):
    """
    Generates a bubble chart of categorized online activities for a specific user.
    """
    user_df = df[df['pid'] == user_id].copy()

    # Aggregate counts by category, excluding noise
    agg = (
        user_df[~user_df['category'].isin(['uncategorized', 'other'])]
        .groupby('category')['pid']
        .count()
        .reset_index(name='count')
    )

    if not agg.empty:
        # Compute circle positions using circlify
        circles = circlify.circlify(
            agg['count'].tolist(),
            show_enclosure=False,
            target_enclosure=circlify.Circle(0, 0, 1.0)
        )

        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis("off")
        ax.set_aspect("equal")

        # Dynamically set limits to prevent bubbles from being cut off
        max_r = 0
        for circle in circles:
            dist = np.sqrt(circle.x**2 + circle.y**2) + circle.r
            if dist > max_r:
                max_r = dist

        ax.set_xlim(-max_r * 1.05, max_r * 1.05)
        ax.set_ylim(-max_r * 1.05, max_r * 1.05)

        cmap = plt.get_cmap("tab20")

        for circle, label, i in zip(circles, agg['category'], range(len(agg))):
            x, y, r = circle.x, circle.y, circle.r
            ax.add_patch(plt.Circle((x, y), r, facecolor=cmap(i % 20), alpha=0.8, edgecolor="white", linewidth=2))

            if r > 0.05:
                ax.text(x, y, label, ha="center", va="center",
                        fontsize=max(8, int(r*80)), fontweight="bold")

        plt.title(f"Online Activity Overview: User {user_id}", fontsize=14, fontweight='bold')
        plt.show()
    else:
        print(f"No categorized data found for User: {user_id}")



In [ ]:
# --- CALL THE FUNCTIONS FOR DEMO ---

target_pid = df_final['pid'].value_counts().idxmax()
print(f"Generating content profile for User: {target_pid}")

plot_top_websites(df_final, target_pid)
plot_activity_bubbles(df_final, target_pid)

####Search Intent (Word Cloud)
While categories tell us *what* type of site a user visits, **Search Queries** tell us *exactly* what they are looking for. By extracting query parameters from search engine URLs (like Google or Bing), we can visualize the user's specific interests.

> **Insight:**
> This visualization completes the "Digital Profile." We now know **when** they browse (Heatmap), **where** they go (Top Domains), **what** they do (Bubble Chart), and finally, **what they think about** (Word Cloud).

**Note**: Since the dataset originates from Germany, we will filter out both English and German "Stop Words" (common words like 'the', 'und', 'der') to ensure only meaningful terms appear.

In [ ]:
def plot_user_wordcloud(df, user_id):
    # Combine English and German stop words to filter out noise
    stop_words = set(get_stop_words("english")) | set(get_stop_words("german"))
    # Add common search-related noise words manually if needed
    # stop_words.update(['google', 'search', 'html', 'index', 'php'])

    # Helper function to extract search terms from URL query parameters
    def extract_search_query(url):
        # Look for 'q=' or 'query=' in the URL string
        match = re.search(r'[?&](q|query)=([^&]+)', str(url))
        if match:
            # Decode URL characters (e.g., %20 to space) and replace '+' with space
            query = unquote(match.group(2)).replace('+', ' ')
            return query
        return ""

    # Filter data for the specific user and extract queries
    user_data = df[df['pid'] == user_id].copy()
    queries = user_data['url'].apply(extract_search_query)

    # Combine all queries into a single string
    all_text = " ".join(queries).lower()

    # Tokenize words: Keep only alphanumeric characters (including German umlauts)
    # Using \wäöüß to support German specific characters
    words = re.findall(r"[\wäöüß]+", all_text)

    # Filter: Remove stop words and very short words (less than 3 chars)
    filtered_words = [w for w in words if w not in stop_words and len(w) >= 3]

    # --- STEP 2: GENERATION ---
    if not filtered_words:
        print(f"No search queries found for User: {user_id}")
        return

    # Count frequencies for better WordCloud control
    word_freq = Counter(filtered_words)

    # Generate the WordCloud object
    # Plasma colormap and high resolution for a professional look
    wc = WordCloud(
        width=1600,
        height=800,
        background_color="white",
        colormap="plasma",
        max_words=100
    ).generate_from_frequencies(word_freq)

    # --- STEP 3: VISUALIZATION ---
    plt.figure(figsize=(16, 8))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off") # Hide axes for the cloud
    plt.title(f"Search Intent Word Cloud (User: {user_id})", fontsize=20, fontweight='bold')
    plt.show()
# Execute the function for our target user
plot_user_wordcloud(df_final, target_pid)


## Part 4: Demographic Perspectives

Data becomes truly powerful when we understand *who* is behind the screen. In this final section, we merge our behavioral features with sociodemographic data (**Age** and **Gender**) to identify group-level trends.

#### Demographic Group Comparison
We will use our **Engineered Features** (like Average Daily Usage and Categories) to answer:
1.  Do different **Age Groups** have different browsing intensities?
2.  How does **Gender** influence the types of content consumed?
3. How does political orientation influence their news consumption?

> **💡 Analytical Tip:**
> Avoid using raw session counts for comparison, as one "power user" can skew the results. Instead, use **Averages per User** within each demographic group to ensure a fair comparison.

In [ ]:
# We merge demo into our feature-rich 'df_final'
df_demographic = pd.merge(df_final, demo_df, on='pid', how='left')

# Calculate Average Daily Intensity per user first
user_intensity = df_demographic.groupby(['pid', 'age_group', 'gender', 'date'])['duration'].sum() / 3600
avg_user_intensity = user_intensity.groupby(['pid', 'age_group', 'gender']).mean().reset_index(name='avg_hours')

plt.figure(figsize=(16, 6))

# Subplot 1: Age Group vs. Browsing Intensity
plt.subplot(1, 3, 1)
sns.boxplot(data=avg_user_intensity, x='age_group', y='avg_hours', hue='age_group', palette='Set2', legend=False)
plt.title("Daily Browsing Intensity by Age Group", fontsize=13, fontweight='bold')
plt.ylabel("Avg Hours per Day")
plt.xlabel("Age Group")

# Subplot 2: Gender & Category Preferences
# We look at the top 5 categories (excluding uncategorized) across genders
top_cats = df_demographic[df_demographic['category'] != 'uncategorized']['category'].value_counts().head(5).index
df_top_cats = df_demographic[df_demographic['category'].isin(top_cats)]

plt.subplot(1, 3, 2)
# Calculate percentage of sessions per category by gender
cat_gender_dist = df_top_cats.groupby(['gender', 'category']).size().unstack(level=0)
cat_gender_dist_pct = cat_gender_dist.div(cat_gender_dist.sum(axis=1), axis=0)

cat_gender_dist_pct.plot(kind='barh', stacked=True, ax=plt.gca(), color=['#ff9999','#66b3ff'])
plt.title("Top 5 Categories: Gender Composition", fontsize=13, fontweight='bold')
plt.xlabel("Proportion of Sessions")
plt.ylabel("")
plt.legend(title='Gender', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

# --- STEP 4: FINAL SUMMARY ---
overall_age_avg = avg_user_intensity.groupby('age_group')['avg_hours'].mean()
print("--- Final Demographic Insights ---")
print(f"Most active age group: {overall_age_avg.idxmax()} ({overall_age_avg.max():.2f} hrs/day)")

In [ ]:
# Merge demographic data
df_demographic = pd.merge(df_final, demo_df, on='pid', how='left')

# 1 to Left (Blue) and 0 to Right (Red)
df_demographic['politics_label'] = df_demographic['politics'].map({1: 'Left', 0: 'Right'})

# Calculate Average Daily Intensity per user
user_intensity = df_demographic.groupby(['pid', 'age_group', 'gender', 'politics_label', 'date'])['duration'].sum() / 3600
avg_user_intensity = user_intensity.groupby(['pid', 'age_group', 'gender', 'politics_label']).mean().reset_index(name='avg_hours')

# Create a large figure for the three-panel analysis
plt.figure(figsize=(24, 7))

# Subplot 1: Age Group vs. Browsing Intensity
plt.subplot(1, 3, 1)
sns.boxplot(data=avg_user_intensity, x='age_group', y='avg_hours', hue='age_group', palette='Set2', legend=False)
plt.title("Daily Browsing Intensity by Age Group", fontsize=13, fontweight='bold')
plt.ylabel("Avg Hours per Day")
plt.xlabel("Age Group")

# Subplot 2: Gender & Category Preferences (Top 5)
plt.subplot(1, 3, 2)
top_cats = df_demographic[df_demographic['category'] != 'uncategorized']['category'].value_counts().head(5).index
df_top_cats = df_demographic[df_demographic['category'].isin(top_cats)]

cat_gender_dist = df_top_cats.groupby(['gender', 'category']).size().unstack(level=0)
cat_gender_dist_pct = cat_gender_dist.div(cat_gender_dist.sum(axis=1), axis=0)

cat_gender_dist_pct.plot(kind='barh', stacked=True, ax=plt.gca(), color=['#ff9999','#66b3ff'])
plt.title("Top 5 Categories: Gender Composition", fontsize=13, fontweight='bold')
plt.xlabel("Proportion of Sessions")
plt.ylabel("")
plt.legend(title='Gender', bbox_to_anchor=(1.05, 1), loc='upper left')

# Subplot 3: Political Orientation & News Media Consumption
plt.subplot(1, 3, 3)

# Filter for news category and top 8 most visited domains
df_news = df_demographic[df_demographic['category'] == 'news'].copy()
top_news_domains = df_news['domain'].value_counts().head(8).index
df_news_filtered = df_news[df_news['domain'].isin(top_news_domains)]

# Calculate normalized proportions within each political group
pol_domain_dist = df_news_filtered.groupby(['politics_label', 'domain']).size().reset_index(name='session_count')
pol_domain_dist['proportion'] = pol_domain_dist.groupby('politics_label')['session_count'].transform(lambda x: x / x.sum())

# Use specific colors: Blue for Left (1), Red for Right (0)
custom_palette = {'Left': '#1f77b4', 'Right': '#d62728'}

sns.barplot(
    data=pol_domain_dist,
    y='domain',
    x='proportion',
    hue='politics_label',
    palette=custom_palette
)


plt.title("News Consumption by Political Orientation", fontsize=13, fontweight='bold')
plt.xlabel("Share of Group's Total News Sessions")
plt.ylabel("")
plt.legend(title='Politics', loc='best')

plt.tight_layout()
plt.show()

# --- STEP 4: FINAL SUMMARY ---
print("--- Final Demographic Insights ---")
overall_age_avg = avg_user_intensity.groupby('age_group')['avg_hours'].mean()
print(f"1. Most active age group: {overall_age_avg.idxmax()} ({overall_age_avg.max():.2f} hrs/day)")

print("\n2. Political News Preference (Top 3 per Group):")
for pol in ['Left', 'Right']:
    subset = df_news[df_news['politics_label'] == pol]
    if not subset.empty:
        top_3 = subset['domain'].value_counts().head(3).index.tolist()
        print(f"   - {pol}: {', '.join(top_3)}")

## Mini Project

Instead of looking at a single user, Please try to analyze the entire population of 50 users.

### What you can do?
1.  **Task 1: Define Your Own Tribe**:
    * Look at the behavioral data and decide on a rule to split the 50 users into 2 or more groups.
    * *Example inspiration*: Are they "Late-Night Browsers"? "Social Media Addicts"?
    * **Visualization**: Use the `plot_segment_rhythms` or `plot_segment_categories` function to see if your tribes actually have different patterns.
2.  **Task 2: Link with Demographics & Politics**
Let's link behavior to **who they are**.
    * Does demographic variables (Gender, Age, Political Orientation) actually result in different browsing habits?


In [ ]:
def plot_segment_rhythms(df, mapping_dict):
    """
    Visualizes the average 24h rhythm for defined segments.
    mapping_dict: {pid: 'Label'}
    """
    df_temp = df.copy()
    df_temp['segment'] = df_temp['pid'].map(mapping_dict)


    rhythm = df_temp.groupby(['segment', 'pid', 'hour']).size().unstack(fill_value=0)

    rhythm_norm = rhythm.div(rhythm.max(axis=1), axis=0)
    avg_rhythm = rhythm_norm.groupby('segment').mean()

    plt.figure(figsize=(12, 5))
    for label in avg_rhythm.index:
        plt.plot(avg_rhythm.columns, avg_rhythm.loc[label], label=label, marker='o', lw=3)

    plt.title("24-Hour Rhythm Comparison Between Your Tribes", fontsize=14, fontweight='bold')
    plt.xlabel("Hour of Day")
    plt.ylabel("Relative Activity Level")
    plt.xticks(range(24))
    plt.legend()
    plt.show()

In [ ]:
def plot_segment_categories(df, mapping_dict, target_categories=None):
    """
    Visualizes actual time distribution. Categories not in target_categories
    are grouped into 'Others' to keep the 100% scale realistic.
    """
    df_temp = df.copy()


    df_temp['segment'] = df_temp['pid'].map(mapping_dict)
    df_temp = df_temp[df_temp['segment'].notna()]


    if target_categories:
        df_temp['category_clean'] = df_temp['category'].apply(
            lambda x: x if x in target_categories else 'Others'
        )
    else:
        df_temp['category_clean'] = df_temp['category']


    cat_stats = df_temp.groupby(['segment', 'category_clean'])['duration'].sum().unstack(fill_value=0)
    cat_stats_pct = cat_stats.div(cat_stats.sum(axis=1), axis=0) * 100



    if 'Others' in cat_stats_pct.columns:
        cols = [c for c in cat_stats_pct.columns if c != 'Others'] + ['Others']
        cat_stats_pct = cat_stats_pct[cols]

    ax = cat_stats_pct.plot(kind='bar', stacked=True, figsize=(12, 7), colormap='Paired')

    plt.title("Actual Content DNA: Share of Total Time per Tribe", fontsize=15, fontweight='bold')
    plt.ylabel("Percentage of Actual Total Time (%)")
    plt.xlabel("Tribe (Grouped by Top Interest)")
    plt.xticks(rotation=0)
    plt.legend(title="Web Categories", bbox_to_anchor=(1.05, 1), loc='upper left')


    for p in ax.patches:
        h = p.get_height()
        if h > 5:
            ax.text(p.get_x() + p.get_width()/2, p.get_y() + h/2, f'{h:.1f}%',
                    ha='center', va='center', fontsize=9, fontweight='bold')

    plt.tight_layout()
    plt.show()



In [ ]:
# ------- WORK AREA----------
# Define your group logic
# Create the mapping {pid: 'Label'}


In [ ]:
df_demographic['used_at'] = pd.to_datetime(df_demographic['used_at'])
df_demographic['hour'] = df_demographic['used_at'].dt.hour

# Define 'Early Birds' vs 'Night Owls'
def categorize_user(user_data):
    avg_hour = user_data['hour'].mean()
    # If their average activity is before 2 PM, call them Early Birds
    if avg_hour < 14:
        return 'Early Birds'
    else:
        return 'Night Owls'

# 3. Create the mapping {pid: 'Label'}

mapping = df_demographic.groupby('pid').apply(categorize_user).to_dict()

# 5. RUN THE VISUALIZATION
plot_segment_rhythms(df_demographic, mapping)

In [ ]:
# Define the 3 Tribes
target_tribe_names = ['social_media', 'news', 'entertainment']

# Assign every user to their top category
user_main_feat = df_final.groupby(['pid', 'category'])['duration'].sum().unstack(fill_value=0).idxmax(axis=1)

# Create the mapping ONLY for those 3 categories

mapping_feat = {
    pid: f"{feat.title()} Tribe"
    for pid, feat in user_main_feat.items()
    if feat in target_tribe_names
}

# target_categories get their own color.
# Everything else (search, shopping, etc.) goes into 'Others'.
plot_segment_categories(
    df_final,
    mapping_feat,
    target_categories=['social_media', 'news', 'entertainment']
)

##Your Own Digital Profile

Do you want to see your own "Browsing profile"? You can apply today's entire pipeline to your personal browsing history!

#### How to do it at home:
1. **Export your history**:
   * Use the Chrome extension ([Export Chrome History](https://chromewebstore.google.com/detail/export-chrome-history/dihloblpkeiddiaojbagoecedbfpifdj?hl=en)).
   * It will generate a CSV file containing columns like `date`, `time`, `url`, and `visitCount`.
2. **Align the data**:
   * Upload your CSV to this notebook.
3. **Run the visualizations**:
   * Re-run these functions using your data.


(if you want, share it tomorrow)


### Pre-reading & Resources on LMM
If you need a refresher on Linear Mixed Models, please refer to:
* **The "Plain English" Guide**: [Introduction to Mixed Effects Models](https://ourcodingclub.github.io/tutorials/mixed-models/) (Coding Club)